In [1]:
# Import packages

import pandas as pd
import math
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
# Read and preview data

df = pd.read_csv('export_yardi_jourentline.csv', lineterminator='\n')
df.head()

,GLCODE,GLNAME,PROPERTY,PROPERTYNAME,UNIT,BUILDING,SPECIALCIRCUMSTANCE,USAGEAMOUNT,USAGETYPE,DATE,PERIOD,DESCRIPTION,CONTROL,REFERENCE,AMOUNT,DEBITCREDIT,BALANCE,REMARKS
0,1110-0000,Cash,ap-mc11,4203 11TH LLC,42031,,,,,09/09/2020,12/01/2020,Webster S8 (t0034597),K-396581,165,8500.0,Credit,1813181.59,First buyout check
1,1110-0000,Cash,ap-mc11,4203 11TH LLC,42032,,,,,09/29/2020,12/01/2020,Webster (t0034598),K-405215,173,2500.0,Credit,1810681.59,
2,1110-0000,Cash,ap-mc11,4203 11TH LLC,42031,,,,,10/27/2020,12/01/2020,Webster S8 (t0034597),K-417819,183,9100.0,Credit,1801581.59,
3,1110-0000,Cash,ap-mc11,4203 11TH LLC,42052,,,,,10/29/2020,12/01/2020,Jones (t0034602),K-420897,187,3500.0,Credit,1798081.59,First Buyout Check
4,1110-0000,Cash,ap-mc11,4203 11TH LLC,42053,,,,,11/02/2020,12/01/2020,Dow (t0034603),K-421957,190,3500.0,Credit,1794581.59,First Buyout Check


In [3]:
# Basic column manipulation

df.columns = df.columns.map(lambda c: c.strip())

df['DATE'] = df['DATE'].map(lambda d: datetime.strptime(d, ' %m/%d/%Y'))
df['PERIOD'] = df['PERIOD'].map(lambda p: datetime.strptime(p, ' %m/%d/%Y'))
df['AMOUNT'] = df['AMOUNT'].map(lambda a: math.ceil(a))

In [4]:
# Find categorical columns

cat_cols = [col for col in df.columns if df[col].dtype == 'object' or df[col].dtype == 'datetime64[ns]']

In [5]:
# Label encode dataset

label_df = df.copy()
label_encoder = LabelEncoder()

for col in cat_cols:
    label_df[col] = label_encoder.fit_transform(df[col])    
    
df = label_df.copy()

In [6]:
# Define X and y

X = df.copy()
X = X.drop(['AMOUNT'], axis=1)

y = df['AMOUNT'].copy()

In [7]:
# Drop columns

X = X.drop(columns=['BUILDING', 'SPECIALCIRCUMSTANCE', 'USAGEAMOUNT', 'USAGETYPE', 'BALANCE', 'CONTROL'])

In [8]:
# Split data into training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [9]:
# Fit/predict model

#max_depth=10, min_child_weight=5.0, and colsample_bytree

model = XGBRegressor(random_state=1, n_jobs=-1, learning_rate=0.92, n_estimators=10000)
model.fit(X_train, y_train, early_stopping_rounds=25, eval_set=[(X_test, y_test)], verbose=False)
training_preds = model.predict(X_train)
test_preds = model.predict(X_test)

print("R^2: ", r2_score(y_test, test_preds))
print("MAE: ", mean_absolute_error(y_test, test_preds))
print("RMSE ", mean_squared_error(y_test, test_preds, squared=False))

R^2:  0.7091026721815714
MAE:  1784.6587073761448
RMSE  24876.061977273086


In [10]:
# Print prediction and actual values

for i in range(len(y_test)):
    print("preds: ", test_preds[i], "actual: ", y_test[i])

preds:  337.50806 actual:  177
preds:  621.1771 actual:  175
preds:  697.9133 actual:  4
preds:  631.95056 actual:  863
preds:  1600.2112 actual:  641
preds:  822.25525 actual:  162
preds:  479.7663 actual:  806
preds:  127.17835 actual:  4
preds:  465.46365 actual:  1602
preds:  1458.7448 actual:  1800
preds:  60.151024 actual:  4
preds:  1092.841 actual:  1365
preds:  233.54945 actual:  4
preds:  535.8438 actual:  1092
preds:  35.49309 actual:  4
preds:  623.28613 actual:  4
preds:  406.8041 actual:  90
preds:  134.85876 actual:  70
preds:  667.7297 actual:  837
preds:  55951.85 actual:  98
preds:  -56.131256 actual:  4
preds:  -404.97202 actual:  90
preds:  1007.57684 actual:  7
preds:  841.2913 actual:  2
preds:  897.703 actual:  34
preds:  476.17657 actual:  875
preds:  1117.3126 actual:  1495
preds:  -61.647224 actual:  143
preds:  86.75602 actual:  4
preds:  694.66034 actual:  844
preds:  -7.110817 actual:  2400
preds:  132.367 actual:  6
preds:  699.2814 actual:  4
preds:  -642

preds:  916.40686 actual:  1434
preds:  2333.478 actual:  2184
preds:  2333.478 actual:  1175
preds:  822.25525 actual:  550
preds:  602.8348 actual:  35
preds:  22.95076 actual:  4
preds:  1521.1964 actual:  2500
preds:  739.9095 actual:  58
preds:  692.0697 actual:  363
preds:  120.681114 actual:  2
preds:  331.2338 actual:  1215
preds:  -258.15613 actual:  35
preds:  1172.581 actual:  1375
preds:  1485.3497 actual:  945
preds:  456.2964 actual:  49
preds:  121.85537 actual:  4
preds:  818.27466 actual:  394
preds:  58.936092 actual:  18
preds:  1310.502 actual:  2145
preds:  1306.9534 actual:  1800
preds:  -200.90958 actual:  150
preds:  389.39316 actual:  5
preds:  1675.5719 actual:  463
preds:  113.984276 actual:  2
preds:  1420.8859 actual:  492
preds:  171.03255 actual:  2
preds:  667.7297 actual:  1645
preds:  602.3291 actual:  439
preds:  535.2099 actual:  40
preds:  75.39819 actual:  149
preds:  602.11456 actual:  2902
preds:  2947.4578 actual:  4727
preds:  -351.71973 actual

preds:  1136.6512 actual:  7
preds:  60.842144 actual:  4
preds:  811.62463 actual:  178
preds:  1234.6636 actual:  1623
preds:  871.94556 actual:  75
preds:  652.3527 actual:  38
preds:  250.73038 actual:  200
preds:  4796.248 actual:  7719
preds:  684.4142 actual:  2
preds:  454.6344 actual:  118
preds:  2676.3208 actual:  19
preds:  4475.366 actual:  289
preds:  352.82373 actual:  2
preds:  351.8792 actual:  3
preds:  123.96502 actual:  2
preds:  765.0111 actual:  600
preds:  559.16315 actual:  46
preds:  2164.2644 actual:  1695
preds:  376.39548 actual:  793
preds:  238.87448 actual:  2230
preds:  737.50867 actual:  512
preds:  132.92494 actual:  12
preds:  123.69445 actual:  29
preds:  71.200035 actual:  464
preds:  326.0978 actual:  247
preds:  184.3905 actual:  8
preds:  1301.6116 actual:  155
preds:  699.73413 actual:  1295
preds:  1379.0259 actual:  350
preds:  134.72678 actual:  262
preds:  773.5091 actual:  40
preds:  1280.5117 actual:  1645
preds:  480.14047 actual:  85
pre

preds:  765.5443 actual:  100
preds:  222.30618 actual:  438
preds:  2042.1086 actual:  2095
preds:  823.3833 actual:  1202
preds:  503.0151 actual:  20
preds:  -4248.9287 actual:  1
preds:  436.40277 actual:  4
preds:  1515.1627 actual:  1395
preds:  939.5189 actual:  299
preds:  757.64307 actual:  719
preds:  420.77695 actual:  4
preds:  729.7329 actual:  402
preds:  60.828487 actual:  106
preds:  1277.0256 actual:  500
preds:  373.87463 actual:  120
preds:  1061.8323 actual:  782
preds:  362.49557 actual:  1355
preds:  1850.8871 actual:  739
preds:  348.51385 actual:  4
preds:  -114.21096 actual:  50
preds:  1233.4252 actual:  1125
preds:  523.3432 actual:  779
preds:  -3946.3052 actual:  1073
preds:  177.27208 actual:  3
preds:  468.76053 actual:  89
preds:  -252.51297 actual:  60
preds:  806.9319 actual:  1195
preds:  -4402.4863 actual:  290
preds:  1417.072 actual:  3146
preds:  3324.2896 actual:  6021
preds:  375.6095 actual:  2095
preds:  704.5185 actual:  4
preds:  1318.1224 a

preds:  958.2174 actual:  250
preds:  387.9354 actual:  516
preds:  616.9356 actual:  802
preds:  465.47662 actual:  1539
preds:  1528.6101 actual:  217
preds:  776.6711 actual:  338
preds:  -115.59031 actual:  75
preds:  357.23593 actual:  4
preds:  -288.7128 actual:  5
preds:  1314.3745 actual:  945
preds:  1808.6215 actual:  401
preds:  -4693.3906 actual:  4
preds:  -4535.047 actual:  6976
preds:  596.68115 actual:  4
preds:  2854.6633 actual:  225
preds:  8619.816 actual:  8313
preds:  635.5084 actual:  235
preds:  445.08878 actual:  4
preds:  107.55547 actual:  35
preds:  -363.1965 actual:  9
preds:  725.9693 actual:  65
preds:  765.23413 actual:  600
preds:  85.33202 actual:  194
preds:  401.58954 actual:  5
preds:  925.0701 actual:  75
preds:  454.48706 actual:  2427
preds:  864.966 actual:  250
preds:  576.5565 actual:  149
preds:  1260.3414 actual:  1350
preds:  915.05023 actual:  50
preds:  -92.385544 actual:  4
preds:  319.72156 actual:  3
preds:  -244.17471 actual:  10
pred

...

preds:  -71.88796 actual:  4
preds:  788.28674 actual:  445
preds:  916.40686 actual:  875
preds:  371.80624 actual:  9
preds:  -4.018097 actual:  2
preds:  854.83435 actual:  900
preds:  97.36002 actual:  1
preds:  13.8088455 actual:  90
preds:  1001.98425 actual:  4
preds:  852.8474 actual:  1920
preds:  274.6339 actual:  159
preds:  554.24255 actual:  273
preds:  207.23392 actual:  2
preds:  798.58057 actual:  70
preds:  -347.79996 actual:  1
preds:  162.01997 actual:  4
preds:  1365.9161 actual:  1539
preds:  1021.7465 actual:  3000
preds:  287.66202 actual:  15
preds:  497.3517 actual:  31
preds:  21.785782 actual:  4
preds:  604.4298 actual:  40
preds:  -224.83525 actual:  4
preds:  74.16093 actual:  150
preds:  1010.85156 actual:  1686
preds:  156.36264 actual:  3
preds:  874.5243 actual:  23
preds:  358.10007 actual:  264
preds:  1124.0089 actual:  1550
preds:  -2549.1665 actual:  164
preds:  156.62361 actual:  793
preds:  -107.84055 actual:  7
preds:  468.76053 actual:  972
pr

preds:  710.1187 actual:  483
preds:  81.17896 actual:  2
preds:  361.2285 actual:  2
preds:  1277.0256 actual:  1000
preds:  989.4664 actual:  4
preds:  54.55955 actual:  2
preds:  -59.268463 actual:  4
preds:  -36.1559 actual:  2
preds:  841.2913 actual:  1407
preds:  67.81401 actual:  3
preds:  524.8965 actual:  704
preds:  1454.4579 actual:  1022
preds:  225.75632 actual:  2
preds:  937.16376 actual:  1129
preds:  -107.84055 actual:  9
preds:  673.4085 actual:  2
preds:  925.0701 actual:  75
preds:  2261.1416 actual:  2279
preds:  -504.99863 actual:  2
preds:  33139.77 actual:  8438
preds:  -1553.276 actual:  39
preds:  1116.0013 actual:  427
preds:  727.93353 actual:  4
preds:  1352.4209 actual:  2090
preds:  500.613 actual:  80
preds:  1324.9674 actual:  2028
preds:  557.3023 actual:  1000
preds:  1837.1183 actual:  31
preds:  1125.9983 actual:  1495
preds:  191.73526 actual:  4
preds:  297.5843 actual:  6
preds:  1114.0288 actual:  1495
preds:  -347.79996 actual:  2
preds:  -68.

preds:  -107.84055 actual:  3
preds:  831.6928 actual:  1518
preds:  339.12762 actual:  15
preds:  937.7771 actual:  2
preds:  181.61588 actual:  135
preds:  194.35712 actual:  95
preds:  180.96548 actual:  1
preds:  360.12616 actual:  1814
preds:  371.52112 actual:  21
preds:  303.92603 actual:  220
preds:  71.38237 actual:  2
preds:  166.87268 actual:  30
preds:  564.7184 actual:  4
preds:  809.5863 actual:  6
preds:  1245.2814 actual:  1250
preds:  935.24036 actual:  1572
preds:  2533.4084 actual:  1296
preds:  1485.3497 actual:  2595
preds:  1904.2721 actual:  2745
preds:  450.08636 actual:  295
preds:  746.8793 actual:  35
preds:  535.2099 actual:  4
preds:  1190.024 actual:  1250
preds:  56.620117 actual:  2
preds:  733.7592 actual:  998
preds:  442.15555 actual:  986
preds:  1783.9786 actual:  1350
preds:  -48.822357 actual:  30
preds:  1273.2781 actual:  1820
preds:  366.3683 actual:  75
preds:  -146.22371 actual:  225
preds:  334.22415 actual:  900
preds:  729.7329 actual:  44

preds:  1367.9988 actual:  1060
preds:  -496.1276 actual:  4
preds:  177.23026 actual:  5
preds:  -137.39267 actual:  6
preds:  465.47662 actual:  2
preds:  -508.2825 actual:  1
preds:  420.77695 actual:  2
preds:  929.577 actual:  1407
preds:  -49.722847 actual:  4
preds:  1260.0302 actual:  1695
preds:  4670.0493 actual:  1800
preds:  -953.84534 actual:  4
preds:  52.677746 actual:  4
preds:  19.330276 actual:  1
preds:  -36.35002 actual:  30
preds:  737.2926 actual:  4
preds:  35.712273 actual:  4
preds:  290.35165 actual:  196
preds:  293.3538 actual:  3
preds:  1341.3938 actual:  1850
preds:  736.7887 actual:  314
preds:  -6369.843 actual:  20
preds:  568.664 actual:  1308
preds:  274.9982 actual:  40
preds:  1435.4225 actual:  1441
preds:  514.377 actual:  26
preds:  524.43524 actual:  5
preds:  662.1493 actual:  20
preds:  353.95203 actual:  2
preds:  -120.51004 actual:  6
preds:  916.19775 actual:  214
preds:  46.59497 actual:  17
preds:  1248.5652 actual:  671
preds:  487.6612

preds:  230.55501 actual:  2
preds:  -928.53033 actual:  8
preds:  591.2891 actual:  500
preds:  0.4071741 actual:  4
preds:  1825.8617 actual:  1012
preds:  -55.080585 actual:  50
preds:  166.20325 actual:  2
preds:  275.61948 actual:  32
preds:  1290.8822 actual:  1777
preds:  852.2971 actual:  801
preds:  963.2726 actual:  1022
preds:  785.48486 actual:  700
preds:  -225.45525 actual:  4
preds:  455.67874 actual:  2
preds:  1419.7643 actual:  265
preds:  1465.4636 actual:  1995
preds:  -110.48024 actual:  4
preds:  661.79675 actual:  4
preds:  832.6506 actual:  2
preds:  506.83972 actual:  937
preds:  1048.0474 actual:  913
preds:  135.94661 actual:  4
preds:  339.5584 actual:  2
preds:  569.36163 actual:  280
preds:  447.38193 actual:  742
preds:  557.3023 actual:  30
preds:  346.91406 actual:  2
preds:  67.81401 actual:  10
preds:  -10.723801 actual:  60
preds:  598.2229 actual:  225
preds:  14425.265 actual:  3865
preds:  34.332302 actual:  2
preds:  -115.706635 actual:  4
preds: